In [1]:
# ====================================================================
# CÉLULA 1: CARREGAR ARTEFATOS, DADOS E FAZER PREVISÕES
# ====================================================================

import pandas as pd
import joblib
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
print("Bibliotecas carregadas.")

# --- 1. CONFIGURAÇÃO DE CAMINHOS ---
BASE_DIR = Path("..") # Caminho relativo ao diretório do notebook
DATA_DIR = BASE_DIR / "data" / "model_inputs"
MODEL_PATH = DATA_DIR / "tourism_model.pkl"
DATA_PATH = DATA_DIR / "model_input.csv"

# --- 2. CARREGAR O PIPELINE TREINADO ---
try:
    pipeline = joblib.load(MODEL_PATH)
    print(f"Pipeline carregado de: {MODEL_PATH}")
    # DIAGNÓSTICO:
    print(f"O modelo espera as seguintes features (ou derivadas): {list(pipeline.feature_names_in_)}")
except FileNotFoundError:
    raise RuntimeError(f"ERRO: Pipeline '{MODEL_PATH}' não encontrado. Execute o 'model_training' primeiro.")

# --- 3. CARREGAR OS DADOS PARA VISUALIZAÇÃO ---
try:
    df = pd.read_csv(DATA_PATH)
    print(f"Dados para visualização carregados: {df.shape[0]} localidades de '{DATA_PATH}'.")
except FileNotFoundError:
    raise RuntimeError(f"ERRO: Arquivo de dados '{DATA_PATH}' não encontrado. Execute o 'data_preparation' primeiro.")

# --- 4. FAZER AS PREVISÕES ---
print("\n🔹 Realizando previsões com o modelo carregado...")
# Passamos o DataFrame inteiro. O pipeline selecionará as colunas que precisa.
predictions = pipeline.predict(df)
df['idh_predito'] = predictions
print("Previsões realizadas com sucesso.")

# --- 5. VERIFICAÇÃO FINAL ---
# ### ALTERAÇÃO CRÍTICA AQUI ###
# Usar 'latitude' e 'longitude', que são os nomes corretos no model_input.csv
colunas_para_mapa = ['poi_nome', 'provincia', 'latitude', 'longitude', 'idh', 'idh_predito']
colunas_em_falta = [col for col in colunas_para_mapa if col not in df.columns]

if not colunas_em_falta:
    print("\n--- Amostra do DataFrame com Previsões ---")
    try:
        from IPython.display import display
        display(df[colunas_para_mapa].head())
    except ImportError:
        print(df[colunas_para_mapa].head())
    print("\n Dados prontos para a visualização na próxima célula.")
else:
    raise ValueError(f"ERRO: As colunas essenciais para o mapa estão em falta: {colunas_em_falta}. Verifique o notebook 'data_preparation'.")

Bibliotecas carregadas.


Pipeline carregado de: ..\data\model_inputs\tourism_model.pkl
O modelo espera as seguintes features (ou derivadas): ['provincia', 'latitude', 'longitude', 'tempmedanual', 'precipitacaoanual', 'ndvi', 'evi', 'ndwi', 'lulccodigo', 'altitude', 'populacao', 'densidadepop', 'pibpercapita', 'taxaurbanizacao', 'empregoturismo', 'distanciaestradaprincipalkm', 'distanciacidadekm', 'acessibilidade', 'tipoviaacesso', 'infraestrutura', 'servicosdisponiveis', 'densidadepopulacionalhabkm']
Dados para visualização carregados: 124 localidades de '..\data\model_inputs\model_input.csv'.

🔹 Realizando previsões com o modelo carregado...
Previsões realizadas com sucesso.

--- Amostra do DataFrame com Previsões ---


,poi_nome,provincia,latitude,longitude,idh,idh_predito
0,Quedas de Calandula,Malanje,-9.074067,16.000316,0.522,0.55004
1,Miradouro da Lua,Luanda,-9.221147,13.090001,0.695,0.70089
2,Museu Kulumbimbi,Zaire,-6.264389,14.245581,0.533,0.54753
3,Reserva Parcial do Namibe,Namibe,-15.766760,12.399914,0.571,0.57462
4,Fortaleza de São Miguel,Luanda,-8.808343,13.223444,0.702,0.71412



 Dados prontos para a visualização na próxima célula.


In [2]:
# ============================================================
# CÉLULA 2: Criar Mapa Interativo com Folium
# ============================================================
import folium

# Centralizar o mapa em Angola
map_center = [-11.2027, 17.8739]
m = folium.Map(location=map_center, zoom_start=6, tiles="CartoDB positron")

# Função para definir a cor do marcador com base no IDH predito
def get_marker_color(idh_predito):
    if idh_predito >= 0.8: return 'darkblue'   # Muito Alto
    elif idh_predito >= 0.7: return 'green'      # Alto
    elif idh_predito >= 0.55: return 'orange'     # Médio
    else: return 'red'        # Baixo

# Adicionar um marcador para cada ponto turístico no DataFrame
for idx, row in df.iterrows():
    # HTML para formatar o popup
    popup_html = f"""
    <h4>{row['poi_nome']}</h4>
    <b>Província:</b> {row['provincia']}<br>
    <hr style='margin: 5px 0;'>
    <b>IDH Real (dados de 2019):</b> {row['idh']:.3f}<br>
    <b>Potencial Predito (IDH):</b> <b style='font-size:1.1em;'>{row['idh_predito']:.3f}</b>
    """
    
    popup = folium.Popup(popup_html, max_width=300)
    
    # Adicionar o marcador ao mapa
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup,
        tooltip=f"{row['poi_nome']} (Potencial: {row['idh_predito']:.3f})",
        icon=folium.Icon(color=get_marker_color(row['idh_predito']), icon='star')
    ).add_to(m)

print("Mapa interativo criado na memória.")

Mapa interativo criado na memória.


In [3]:
# Exibir mapa dentro do notebook
# ============================================================
#  Exibir Mapa Interativo
# ============================================================

# Ao executar esta célula, o mapa interativo será renderizado abaixo.
m

In [4]:
# Adicionar camadas auxiliares

# Adicionar um controle de camadas para ligar/desligar diferentes tiles
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Terrain', attr='Stamen Terrain').add_to(m)
folium.LayerControl().add_to(m)

print(" Camadas auxiliares adicionadas ao mapa.")

 Camadas auxiliares adicionadas ao mapa.


In [5]:
# Entrega esperada: outputs/mapa_turismo.html/index.html
import os
# ============================================================
# 5️ Salvar o Mapa como um Ficheiro HTML
# ============================================================

# Definir o caminho e o nome do arquivo de saída
OUTPUT_DIR = "../outputs"
MAP_PATH = os.path.join(OUTPUT_DIR, "index.html")

# Criar o diretório se ele não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Salvar o objeto do mapa em um ficheiro HTML
m.save(MAP_PATH)

print(f" Mapa salvo com sucesso em: {MAP_PATH}")

# Confirmação da entrega
if os.path.exists(MAP_PATH):
    print(f"\n Entrega confirmada: o arquivo '{os.path.basename(MAP_PATH)}' existe no diretório '{OUTPUT_DIR}'.")
else:
    print(f"\n Atenção: o arquivo do mapa não foi encontrado no local esperado.")

 Mapa salvo com sucesso em: ../outputs\index.html

 Entrega confirmada: o arquivo 'index.html' existe no diretório '../outputs'.
